## Imports

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import requests
import datetime
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

## Get Data from Greek Covid-19 API

In [8]:
url = "https://covid-19-greece.herokuapp.com/all"
response = requests.get(url)
jsondata = response.json()

In [9]:
jsondata.keys()

dict_keys(['cases'])

In [10]:
jsondata['cases'][0]

{'confirmed': 0, 'date': '2020-01-22', 'deaths': 0}

## Read Data with Pandas and Preprocessing

In [11]:
columns=['confirmed','date','deaths']
df = pd.DataFrame.from_dict(jsondata['cases'])
data = []
for x in jsondata['cases']:
    data.append([x['confirmed'],x['date'],x['deaths']])

df = pd.DataFrame(data,columns=columns)
df['daily_cases'] = df['confirmed'] - df['confirmed'].shift(1)
df['daily_deaths'] = df['deaths'] - df['deaths'].shift(1)
df = df.dropna()
df['daily_cases'] = pd.to_numeric(df['daily_cases'],downcast='integer')
df['daily_deaths'] = pd.to_numeric(df['daily_deaths'],downcast='integer')
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')

## Plot of Daily Cases with Moving Averages

In [16]:
x = df.copy()
x['SMA5'] = df['daily_cases'].rolling(window=5).mean()
x['SMA10']= df['daily_cases'].rolling(window=10).mean()
x = x.dropna(axis=0)
fig = go.Figure()

fig.add_trace(
    go.Line(
        x=x['date'],
        y=x['daily_cases'],
        name='Daily Cases'
    ))

fig.add_trace(
    go.Line(
        x=x['date'],
        y=x['SMA5'],
        name='5 Day Moving Average'
    ))

fig.add_trace(
    go.Line(
        x=x['date'],
        y=x['SMA10'],
        name='10 Day Moving Average'
    ))
fig['layout'].update(height = 600, width = 1050,xaxis=dict(
      tickangle=-30
    ))
fig.show()


## Get Tests from Greek COVID-19 API

In [17]:
url = "https://covid-19-greece.herokuapp.com/total-tests"
response = requests.get(url)
jsondata = response.json()

In [18]:
jsondata.keys()

dict_keys(['total_tests'])

In [19]:
jsondata['total_tests'][0]

{'date': '2020-02-26', 'rapid-tests': 0, 'tests': None}

In [47]:
columns=['date','rapid-tests','tests']
tests = pd.DataFrame.from_dict(jsondata['total_tests'])
data_ = []
for k in jsondata['total_tests']:
    data_.append([k['date'],k['rapid-tests'],k['tests']])

tests = pd.DataFrame(data_,columns=columns)
tests['tests'] = tests['tests'].replace(np.nan,0).astype(int)
tests['date'] = pd.to_datetime(tests['date'],format='%Y-%m-%d')
tests

,date,rapid-tests,tests
0,2020-02-26,0,0
1,2020-02-27,0,0
2,2020-02-28,0,0
3,2020-02-29,0,0
4,2020-03-01,0,0
...,...,...,...
613,2021-11-01,19088035,7002201
614,2021-11-02,19485435,7024209
615,2021-11-03,19701192,7040861
616,2021-11-04,19887313,7057798


Because the "TESTS" dataset started getting data at February 26th, we drop the first few days since they are insignificant.

In [48]:
cases = df[df.date>= '2020-02-26']
cases

,confirmed,date,deaths,daily_cases,daily_deaths
35,1,2020-02-26,0,1,0
36,3,2020-02-27,0,2,0
37,4,2020-02-28,0,1,0
38,4,2020-02-29,0,0,0
39,7,2020-03-01,0,3,0
...,...,...,...,...,...
649,747595,2021-11-01,15990,5425,52
650,754451,2021-11-02,16050,6856,60
651,760592,2021-11-03,16109,6141,59
652,767376,2021-11-04,16151,6784,42


In [49]:
print(cases.shape)
print(tests.shape)

(619, 5)
(618, 3)


## MERGE the two datasets

In [93]:
comple = cases.merge(tests,how='left',on='date')

In [94]:
comple['daily_rapid'] = comple['rapid-tests'] - comple['rapid-tests'].shift(1)
comple['daily_tests'] = comple['tests'] - comple['tests'].shift(1)
comple = comple[comple['daily_tests'] >= 0]
comple = comple[comple['daily_rapid'] >= 0]
comple.dropna(inplace=True)
comple

,confirmed,date,deaths,daily_cases,daily_deaths,rapid-tests,tests,daily_rapid,daily_tests
1,3,2020-02-27,0,2,0,0.0,0.0,0.0,0.0
2,4,2020-02-28,0,1,0,0.0,0.0,0.0,0.0
3,4,2020-02-29,0,0,0,0.0,0.0,0.0,0.0
4,7,2020-03-01,0,3,0,0.0,0.0,0.0,0.0
5,7,2020-03-02,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
614,747595,2021-11-01,15990,5425,52,19088035.0,7002201.0,76503.0,6090.0
615,754451,2021-11-02,16050,6856,60,19485435.0,7024209.0,397400.0,22008.0
616,760592,2021-11-03,16109,6141,59,19701192.0,7040861.0,215757.0,16652.0
617,767376,2021-11-04,16151,6784,42,19887313.0,7057798.0,186121.0,16937.0


### Calculate positivity rate

In [101]:
comple['positivity'] = comple['daily_cases']/(comple['daily_rapid'] + comple['daily_tests'])
comple['positivity'] = comple['positivity'].replace(np.nan,0)
comple['positivity'] = comple['positivity'].replace(np.inf,0)
comple['positivity'] = 100*comple['positivity']
comple

,confirmed,date,deaths,daily_cases,daily_deaths,rapid-tests,tests,daily_rapid,daily_tests,positivity
1,3,2020-02-27,0,2,0,0.0,0.0,0.0,0.0,0.000000
2,4,2020-02-28,0,1,0,0.0,0.0,0.0,0.0,0.000000
3,4,2020-02-29,0,0,0,0.0,0.0,0.0,0.0,0.000000
4,7,2020-03-01,0,3,0,0.0,0.0,0.0,0.0,0.000000
5,7,2020-03-02,0,0,0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
614,747595,2021-11-01,15990,5425,52,19088035.0,7002201.0,76503.0,6090.0,6.568353
615,754451,2021-11-02,16050,6856,60,19485435.0,7024209.0,397400.0,22008.0,1.634685
616,760592,2021-11-03,16109,6141,59,19701192.0,7040861.0,215757.0,16652.0,2.642325
617,767376,2021-11-04,16151,6784,42,19887313.0,7057798.0,186121.0,16937.0,3.340917
